In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

 preprocessing the text
 

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

applying pad sequences to make sentence length equal


creating model for train set


In [ ]:
embedding_vector_features=40
model=Sequential()

importing dataset

In [ ]:
! pip install -q kaggle
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Saving kaggle.json to kaggle (5).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions download -c fake-news

401 - Unauthorized


In [ ]:
!unzip train.csv.zip
!unzip test.csv.zip

unzip:  cannot find or open train.csv.zip, train.csv.zip.zip or train.csv.zip.ZIP.
unzip:  cannot find or open test.csv.zip, test.csv.zip.zip or test.csv.zip.ZIP.


In [ ]:
import pandas as pd
data= pd.read_csv("train.csv")
data.head(10)

FileNotFoundError: ignored

removing NaN values from the dataset


In [ ]:
data = data.dropna()
data.reset_index(inplace=True)

extracting dependent and independent variables from data

In [ ]:
data.drop('label',axis=1)
y=data["label"]


checking the shape of x and y


In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
messages=X.copy()
ps= PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]","",messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)
  


In [ ]:
vocab_size= 5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus]

In [ ]:
sent_length = 20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

creating model for train set


In [ ]:
embedding_vector_features=40
model= Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test= train_test_split(X_final, y_final, test_size=0.33, random_state= 42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)